In [10]:
import numpy as np
from copy import copy
import pandas as pd
import matplotlib.pyplot as plt
from brdfFile import brdfFile
from kernels import kernelBRDF
from gort import gort, geom

from satelliteGeometry import *
startDate = datetime(2020,1,1,0,0,0,0,None)
alt=0.
days=365

def geom_list_from_brdfFile(b):
    geom_list = []
    for i in range(b.nAngles):
        geom_list.append(geom(vza=b.vza_arr[i], vaa=b.vaa_arr[i], sza=b.sza_arr[i], saa=b.saa_arr[i]))
    return geom_list

def add_obs_brfs_to_kernelBRDF(brfs,k):

    for i in range( k.nAngles ):
        for j in range( k.nWavelengths ):
            k.brfObs[i][j]=brfs[i][j]

    return k

In [11]:
# make dictionary of latitudes/longitudes
lat_lons = {
            'lon_-120': [-120.0, (40.0,50.0,60.0,70.0)],
            'lon_-60': [-60.0, (-30.0,-20.0,-10.0,0.0)],
            'lon_0': [0.0, (10.0,20.0,30.0,40.0,50.0)],
            'lon_60': [60.0, (30.0,40.0,50.0,60.0)],
            'lon_120': [120.0, (30.0,40.0,50.0,60.0,70.0)]
            }
latlon_keys = list(lat_lons.keys())

In [12]:
# get geometry and datetime files
#Lat0.0_Lon-60.0_overpasses_stepfine0.10s_stepcoarse5.00s_mindistkm50.00km
filefolder='/home/users/ndouglas/TRUTHS/Geometries/TRUTHS/truths_orbit_for_reading-20250908T114401Z-1-001/truths_orbit_for_reading/'
filepathext='_overpasses_stepfine0.10s_stepcoarse5.00s_mindistkm50.00km.csv'

for lonref in latlon_keys:
    longitude=lat_lons[lonref][0]
    print(longitude)
    for latitude in lat_lons[lonref][1]:
        print(latitude)
        albedo_filepath='/home/users/ndouglas/TRUTHS/albedos/LAT'+str(latitude)+'LON'+str(longitude)
        #if not os.path.exists(albedo_filepath):
        #    os.makedirs(albedo_filepath)
        TRUTHS_geometries_filepath=filefolder+'Lat'+str(latitude)+'_Lon'+str(longitude)+filepathext
        df1 = pd.read_csv(TRUTHS_geometries_filepath)
        list_vza = np.array(df1['view_zenith_angle_deg'].tolist())
        list_solaa = np.array(df1['solar_azimuth_deg'].tolist())
        list_solea = np.array(df1['solar_elevation_deg'].tolist())
        #list_sataa = np.array(df1['satellite_azimuth_deg'].tolist())
        #list_vaa_temp = list_sataa - list_solaa
        #list_vaa = [min(np.abs(element), 360 - np.abs(element)) for element in list_vaa_temp]
        list_vaa = np.array(df1['satellite_azimuth_deg'].tolist())
        list_solza = [90 - element for element in list_solea]
        TRUTHS_brdf_filename='/home/users/ndouglas/TRUTHS/Geometries/TRUTHS/TRUTHSgeometries/TRUTHSgeomsLAT'+str(latitude)+'LON'+str(longitude)+'.brdf'
        
        list_datetimes = np.array(df1['datetime'].str[:16].tolist())
        TRUTHS_geometries_list = pd.DataFrame(
            {'datetime': list_datetimes,
             'viewing_zenith_angle': list_vza,
             'viewing_azimuth_angle': list_vaa,
             'solar_zenith_angle': list_solza,
             'solar_azimuth_angle': list_solaa
            })
        TRUTHS_geometries_list=TRUTHS_geometries_list[TRUTHS_geometries_list["solar_zenith_angle"]<70.0]
        TRUTHS_geometries_list = TRUTHS_geometries_list.reset_index()
        TRUTHS_geometries_list = TRUTHS_geometries_list.rename(columns={"index": "mission"})
        TRUTHS_geometries_list["mission"] = "TRUTHS"
        
        list_datetimes_sent=[]
        list_vza_sent=[]
        list_vaa_sent=[]
        list_sza_sent=[]
        list_saa_sent=[]
          
        geomList=getSentinel2Geometry(startDate,days,latitude,longitude,mission="Sentinel-2a",alt=alt) 
        #print('original geomList',np.shape(geomList))
        for g in geomList:
            if g.sza < 70.0:
            #print(g.vza, g.vaa, g.sza, g.saa)
                list_datetimes_sent.append(g.datetime.strftime("%Y-%m-%d %H:%M"))
                list_vza_sent.append(g.vza)
                list_vaa_sent.append(g.vaa)
                list_sza_sent.append(g.sza)
                list_saa_sent.append(g.saa)
        Sentinel_geometries_list = pd.DataFrame(
            {'datetime': list_datetimes_sent,
             'viewing_zenith_angle': list_vza_sent,
             'viewing_azimuth_angle': list_vaa_sent,
             'solar_zenith_angle': list_sza_sent,
             'solar_azimuth_angle': list_saa_sent
            })
        Sentinel_geometries_list = Sentinel_geometries_list.reset_index()
        Sentinel_geometries_list = Sentinel_geometries_list.rename(columns={"index": "mission"})
        Sentinel_geometries_list["mission"] = "Sentinel2"

        geometries_list=pd.concat([TRUTHS_geometries_list, Sentinel_geometries_list], ignore_index=True)
        #geometries_list.to_csv(albedo_filepath+'/LAT'+str(latitude)+'LON'+str(longitude)+'_geometries.csv')
        geometries_list.to_csv('/home/users/ndouglas/TRUTHS/albedos/geometries/LAT'+str(latitude)+'LON'+str(longitude)+'_geometries.csv')

        #TRUTHS_datetimes_temp = np.array(df1['datetime'].str[:16].tolist())
        #list_solea = np.array(df1['solar_elevation_deg'].tolist())
        #TRUTHS_datetimes=TRUTHS_datetimes_temp[np.where(list_solea>20.0)]
        #print(TRUTHS_datetimes)
        #with open('/home/users/ndouglas/TRUTHS/datetimes/TRUTHS/TRUTHSdatetimesLAT'+str(latitude)+'LON'+str(longitude),"w") as f:
        #    for m in range(0,len(TRUTHS_datetimes)):
        #        f.write("%s \n"%TRUTHS_datetimes[m])        

        #Sentinel_datetimes=[]
        #geomList=getSentinel2Geometry(startDate,days,latitude,longitude,mission="Sentinel-2a",alt=alt)
        #for g in geomList:
        #    if g.sza < 70.0:
        #        Sentinel_datetimes.append(g.datetime.strftime("%Y-%m-%d %H:%M:%S"))
        #with open('/home/users/ndouglas/TRUTHS/datetimes/Sentinel2/Sentinel2datetimesLAT'+str(latitude)+'LON'+str(longitude),"w") as f:
        #    for m in range(0,len(Sentinel_datetimes)):
        #        f.write("%s \n"%Sentinel_datetimes[m])        


-120.0
40.0
50.0
60.0
70.0
-60.0
-30.0
-20.0
-10.0
0.0
0.0
10.0
20.0
30.0
40.0
50.0
60.0
30.0
40.0
50.0
60.0
120.0
30.0
40.0
50.0
60.0
70.0


In [11]:
# get albedo and brf files
LAI=5.0
PCC=0.7

feature='BRFs'

for lonref in latlon_keys:
    longitude=lat_lons[lonref][0]
    for latitude in lat_lons[lonref][1]:
        albedo_filepath='/home/users/ndouglas/TRUTHS/albedos/LAT'+str(latitude)+'LON'+str(longitude)

        TRUTHS_brdf_filename='/home/users/ndouglas/TRUTHS/Geometries/TRUTHS/TRUTHSgeometries/TRUTHSgeomsLAT'+str(latitude)+'LON'+str(longitude)+'.brdf'      

        k_TRUTHS=kernelBRDF( )
        k_TRUTHS.readBRDF(TRUTHS_brdf_filename)
        geom_list=geom_list_from_brdfFile(k_TRUTHS)
        #print(geom_list)

        g=gort() 
        g.params["LAI"]=LAI
        g.params["PCC"]=PCC
        g.params["HB"]=1.1
        g.params["BR"]=1.0
        g.wavl=copy(k_TRUTHS.wavelengths)    
        gort_brfs, comp, enrg = g.run(geom_list, verbose=False)
        #print(gort_brfs)
        gort_alb=enrg[:,::3]

        if feature == 'albedos':
            k_TRUTHS=add_obs_brfs_to_kernelBRDF(gort_alb,k_TRUTHS)
        else:
            k_TRUTHS=add_obs_brfs_to_kernelBRDF(gort_brfs,k_TRUTHS)
        #print(k_TRUTHS.brfObs)

        TRUTHS_albedos_list = pd.DataFrame(k_TRUTHS.brfObs, columns=k_TRUTHS.wavelengths)
        TRUTHS_albedos_list = TRUTHS_albedos_list.reset_index()
        TRUTHS_albedos_list = TRUTHS_albedos_list.rename(columns={"index": "mission"})
        TRUTHS_albedos_list["mission"] = "TRUTHS"

        Sentinel_brdf_filename='/home/users/ndouglas/TRUTHS/Geometries/Sentinel/SentinelGeometries/SentinelGeomsLAT'+str(latitude)+'LON'+str(longitude)+'.brdf'      

        k_Sentinel=kernelBRDF( )
        k_Sentinel.readBRDF(Sentinel_brdf_filename)
        geom_list=geom_list_from_brdfFile(k_Sentinel)

        gort_brfs, comp, enrg = g.run(geom_list, verbose=False)
        gort_alb=enrg[:,::3]

        if feature == 'albedos':
            k_Sentinel=add_obs_brfs_to_kernelBRDF(gort_alb,k_Sentinel)
        else:
            k_Sentinel=add_obs_brfs_to_kernelBRDF(gort_brfs,k_Sentinel)

        Sentinel_albedos_list = pd.DataFrame(k_Sentinel.brfObs, columns=k_Sentinel.wavelengths)
        Sentinel_albedos_list = Sentinel_albedos_list.reset_index()
        Sentinel_albedos_list = Sentinel_albedos_list.rename(columns={"index": "mission"})
        Sentinel_albedos_list["mission"] = "Sentinel2"

        albedos_list=pd.concat([TRUTHS_albedos_list, Sentinel_albedos_list], ignore_index=True)
        if feature == 'albedos':
            albedos_list.to_csv(albedo_filepath+'/LAT'+str(latitude)+'LON'+str(longitude)+'LAI'+str(LAI)+'PCC'+str(PCC)+'_albedos.csv')
        else:
            albedos_list.to_csv(albedo_filepath+'/LAT'+str(latitude)+'LON'+str(longitude)+'LAI'+str(LAI)+'PCC'+str(PCC)+'_BRFs.csv')